# LastMile Instrumentor for LangChain


In this notebook, we showcase how to use the **LastMile Tracing SDK** to auto-instrument tracing for a LangChain "Plan-and-Execute" agent. With tracing automatically setup, you can easily debug your application using LastMile's RAG Debugger.

The LangChain "Plan-and-Execute" agent accomplishes an objective by first planning what to do and then executing the sub-tasks. LastMile autoinstrumentation is used to trace and monitor the execution of the agent.

## Notebook Outline
* [Step 1: Setup](#setup)
* [Step 2: Configure the LastMile Instrumentor](#step2)
* [Step 3: Define Tools for LangChain Agent](#step3)
* [Step 4: Run example with LangChain Agent](#step4)
* [Step 5: View Trace Data in RAG Debugger](#step5)

<a name="setup"></a>
# Step 1: Setup

To begin, we need to install a few packages including langchain and lastmile-eval.

In [ ]:
# !pip install langchain
# !pip install langchain_community
# !pip install langchain_core
# !pip install langchain_experimental
# !pip install langchain_openai
# !pip install wikipedia
# !pip install numexpr
# !pip install lastmile_eval
# !pip install "tracing-auto-instrumentation[langchain]" --upgrade

Import the necessary libraries for this example.

In [ ]:
from langchain.chains import LLMMathChain
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import Tool
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)
from langchain_openai import ChatOpenAI, OpenAI

Before we start this tutorial, we need the following tokens/keys:

* LastMile AI API Token: Go to the [LastMile Settings page](https://lastmileai.dev/settings?page=tokens). You will need to first create a LastMile AI account.
* OpenAI API Key: Go to [OpenAI API Keys page](https://platform.openai.com/account/api-keys) to create and access your OpenAI API Key.

You can either set these explicitly (uncomment the lines below), or save them in a `.env` file within this project directory.


In [ ]:
import dotenv
dotenv.load_dotenv()

# import os
# os.environ['OPENAI_API_KEY'] =  os.getenv('OPENAI_API_KEY')
# os.environ['LASTMILE_API_TOKEN'] =  os.getenv('LASTMILE_API_TOKEN')

<a name="step2"></a>

# Step 2: Configure the LastMile Instrumentor

Next, we create an instance of `LangChainInstrumentor` with a project name. The `instrument()` method is called to instrument the code for tracing and monitoring.

In [ ]:
from tracing_auto_instrumentation.langchain import LangChainInstrumentor

# Create an instance of LangChainInstrumentor and instrument the code
instrumentor = LangChainInstrumentor(project_name="LangChain-Example")
instrumentor.instrument()

<a name="step3"></a>

# Step 3: Define Tools for Langchain Agent

The tools that the agent will use for executing sub-tasks are defined. In this example, we have two tools:
- "Wikipedia" tool: Uses the `WikipediaAPIWrapper` to look up information on Wikipedia.
- "Calculator" tool: Uses the `LLMMathChain` to perform mathematical calculations.

In [ ]:
# Define the tools that the agent will use for executing sub-tasks
wikipedia = WikipediaAPIWrapper()
llm = OpenAI(temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="useful for when you need to look up information on Wikipedia",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to perform mathematical calculations",
    ),
]


<a name="step4"></a>

# Step 4: Run example with LangChain Agent

Let's try an example with our LangChain agent. We will ask a simple question "What is hte population of the capital city of the country where the Eiffel Tower is located? Calculate the square root of that number."

The agent needs to look up the population (using Wikipedia) and then also make a calculation.

In [ ]:
model = ChatOpenAI(temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor)

# ## Run Example

# Run the agent with a sample question
question = "What is the population of the capital city of the country where the Eiffel Tower is located? Calculate the square root of that number."
result = agent.run(question)

In [ ]:
print(result)

<a name="step5"></a>

# Step 5: View Trace Data in RAG Debugger
You can view the results in the RAG Workbench UI. Here's how to access the UI:

1. Install the UI package by running the following command in your terminal:

    `pip install "lastmile-eval[ui]”`

2. Export your LASTMILE_API_TOKEN by running the following command in your terminal, replacing <your-api-token> with your actual API token:

    `export LASTMILE_API_TOKEN="<your-api-token>”`

3. Run the following command in your terminal:

    `rag-debug launch`

Once you've completed these steps, click on the url provided to open the RAG Workbench UI. 

Navigate to the 'Traces' Page where you see all the Traces listed under this Project (top-right corner to choose Project).

<img width="973" alt="Screenshot 2024-05-28 at 12 06 53 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/a2ad1630-dc62-43a6-8d81-cfa335fe8fb1"/>


Let's click into the Trace.

<img width="973" alt="Screenshot 2024-05-28 at 12 07 04 PM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/01bb1693-a79c-40a6-962e-15446cbe03c6"/>


Here we can see all the spans auto-generated for us. This can help us debug and pinpoint issues in our application, especially if we add additional logging on top of the auto-instrumentor.